## Scraping

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import time
import random as ran
from selenium import webdriver

In [2]:
def get_reviews(r):
    reviews = {}
    try:
        reviews['user_name'] = r.select('[class*=css-166la90]')[0].get_text() 
        reviews['user_url'] = r.select('[class*=css-166la90]')[0]['href']
        reviews['user_city'] = r.select('[class*=css-n6i4z7]')[0].get_text() 
        reviews['user_rating'] = r.select('[aria-label*=rating]')[0]['aria-label']
        reviews['user_review'] = r.select('[class*=raw__]')[0].get_text().strip()
    except:
        reviews['user_name'] = np.nan
        reviews['user_url'] = np.nan
        reviews['user_city'] = np.nan
        reviews['user_rating'] = np.nan
        reviews['user_review'] = np.nan
    return reviews

In [3]:
def get_biz(biz_url):
    df = pd.DataFrame()
    p = 0

    #biz_url = 'https://www.yelp.com/biz/damas-montr%C3%A9al'
    driver = webdriver.Chrome('chromedriver')
    driver.get(biz_url)
    time.sleep(ran.randint(2, 5))
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    revs = soup.select('[class*=review__]') 

    # Get reviews from a restaurant
    while revs != []: 

        for r in revs:
            gr = get_reviews(r)
            #print(gr)
            df = pd.concat([df, pd.DataFrame.from_records(gr, index=[0])])

        p += 20
        url = f'{biz_url}?start={p}'
        
        driver.get(url)
        time.sleep(ran.randint(3, 5))
        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        revs = soup.select('[class*=review__]') 


    time.sleep(ran.randint(0, 10))
    driver.quit()
    return df

In [ ]:
cnt = 0
biz = pd.read_csv('data/businesses_mtl.csv')

# This will output a csv file for each restaurant
for b in biz.url[0:]:

    dx = get_biz(b)
    dx = dx.reset_index(drop=True)
    dx = dx[['user_url', 'user_name', 'user_city', 'user_rating', 'user_review']]
    cnt +=1
    dx.to_csv('data/raw/' + str(cnt) + '_biz_reviews.csv', index=False)

In [ ]:
### Testing code ###
'''
p = 20
df = pd.DataFrame()
biz_url = 'https://www.yelp.com/biz/damas-montr%C3%A9al'

url = f'{biz_url}?start={p}'
driver = webdriver.Chrome('chromedriver')
driver.get(url)
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')
revs = soup.select('[class*=review__]')
#driver.quit()

for r in revs:
    reviews = {}
    reviews['user_name'] = r.select('[class*=css-166la90]')[0].get_text()
    reviews['user_url'] = r.select('[class*=css-166la90]')[0]['href']
    reviews['user_city'] = r.select('[class*=css-n6i4z7]')[0].get_text()
    reviews['user_rating'] = r.select('[aria-label*=rating]')[0]['aria-label']
    reviews['review'] = r.select('[class*=raw__]')[0].get_text().strip()
    df = pd.concat([df, pd.DataFrame.from_records(reviews, index=[0])])
df
'''